In [1]:
library(MASS)
library(tidyverse) 
library(tidytext)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ dplyr::select() masks MASS::select()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
master_dir = file.path(list.files('../input', full.names=TRUE), 'youtube-personality')
directory_content = list.files(master_dir, full.names = TRUE)

# Path to the transcripts directory with transcript .txt files
path_to_transcripts = directory_content[2] 

# .csv filenames (see output above)
AudioVisual_file    = directory_content[3]
Gender_file         = directory_content[4]
Personality_file    = directory_content[5]

# 1. Importing the data

We will import the following:

- Transcripts
- Audiovisual information
- Personality scores
- Gender

## 1.1 Description of the YouTube Personality Dataset

The YouTube Personality dataset comprises an array of behavioral features, speech transcriptions, and personality impression scores for a group of 404 YouTube vloggers. These vloggers present themselves in front of a webcam while discussing a wide range of topics, encompassing personal matters, politics, movies, books, and more. Notably, there are no content-related restrictions, and the language employed in the videos is both natural and diverse.


## 1.2 Objective: Predicting Missing Personality Trait Scores

Our primary objective centers around a subset of this dataset: specifically, 80 out of the 404 vloggers have incomplete personality trait scores. Leveraging text analysis techniques, we aim to predict and fill in these missing personality scores, contributing to a more comprehensive understanding of the vloggers' personalities.

## 1.3 Big Five Personality Traits 

### Extraversion:
Extraversion is characterized by a high level of energy in social interactions, assertiveness, and a propensity for seeking new sensations. People with high extraversion are often described as "life of the party," talkative, and natural leaders within a group.

### Agreeableness:
Agreeableness reflects an individual's desire to maintain harmonious relationships with others. Key attributes include humility, trustworthiness, and patience. Those high in agreeableness may be hesitant to express opinions that conflict with others.

### Conscientiousness:
Conscientious individuals exhibit discipline, a preference for organized tasks, and a strong commitment to doing what is morally right. Typical behaviors for conscientious individuals include never cheating and ensuring the completion of tasks.

### Emotionality (Neuroticism):
Emotionality, also known as Neuroticism, measures the extent to which a person experiences negative emotions and how these emotions impact their well-being. Individuals high in emotionality often grapple with depression, anxiety, or self-consciousness.

### Openness to Experience:
Openness to Experience captures a person's curiosity and interest in exploring new ideas, values, and behaviors.

Source:
Huntington, C. (n.d.). Big Five Personality Traits: Definition & Theory. Retrieved from https://www.berkeleywellbeing.com/big-five-personality-traits.html
 
 


## 1.4 Importing transcripts

In [3]:
transcript_files = list.files(path_to_transcripts, full.names = TRUE) 

# Extract vlogger IDs
vlogId = basename(transcript_files)
vlogId = str_replace(vlogId, pattern = ".txt$", replacement = "")

To include features extracted from the transcript texts you will have to read the text from files and store them in a data frame. For this, you will need the full file paths as stored in `transcript_files`.

In [4]:
transcripts_df = tibble(
    
    # vlogId connects each transcripts to a vlogger
    vlogId=vlogId,
    
    # Read the transcript text from all file and store as a string
    TEXT = map_chr(transcript_files, ~ paste(readLines(.x), collapse = "\\n")), 
    
    # `filename` keeps track of the specific video transcript
    filename = transcript_files
)

Warning message in readLines(.x):
“incomplete final line found on '../input/bda-2023-profiling-personality/youtube-personality/transcripts/VLOG11.txt'”


## 1.5 Import personality and gender scores

Gender is a useful predictor for personality scores as previous research shows. Schmitt et al. (2008), for instance, conducted a cross-cultural analysis invovling data from 55 different nations (total sample size of 17,637 individuals). They found that women tended to report higher levels of neuroticism, extraversion, agreeableness, and conscientiousness compared to men in most of the nations studied. 

In [5]:
# Import the Personality scores
pers_df = read_delim(Personality_file, delim = " ")

head(pers_df, n = 3)

Rows: 324 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: " "
chr (1): vlogId
dbl (5): Extr, Agr, Cons, Emot, Open

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


vlogId,Extr,Agr,Cons,Emot,Open
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
VLOG1,4.9,3.7,3.6,3.2,5.5
VLOG3,5.0,5.0,4.6,5.3,4.4
VLOG5,5.9,5.3,5.3,5.8,5.5


Gender info is stored in a separate `.csv` which is also delimited with a space. This file doesn't have column names, so we have to add them ourselves:

In [6]:
gender_df = read.delim(Gender_file, head=FALSE, sep=" ", skip = 2)

# Add column names
names(gender_df) = c('vlogId', 'gender')


head(gender_df, n = 3)

,vlogId,gender
,<chr>,<chr>
1,VLOG3,Female
2,VLOG5,Male
3,VLOG6,Male


## 1.6 Merging the gender and personality dataframes

Use left_join() to merge all the information in a single tidy data frame.

In [7]:
vlogger_df = left_join(gender_df, pers_df, by='vlogId')
head(vlogger_df) # VLOG8 has missing personality scores: those should be predicted

,vlogId,gender,Extr,Agr,Cons,Emot,Open
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,VLOG3,Female,5.0,5.0,4.6,5.3,4.4
2,VLOG5,Male,5.9,5.3,5.3,5.8,5.5
3,VLOG6,Male,5.4,4.8,4.4,4.8,5.7
4,VLOG7,Male,4.7,5.1,4.4,5.1,4.7
5,VLOG8,Female,NA,NA,NA,NA,NA
6,VLOG9,Female,5.6,5.0,4.0,4.2,4.9


We leave the `transcripts_df` data frame seperate for now, because you will first have to extract features from the transcripts first. Once you have those features in a tidy data frame, including a `vlogId` column, you can refer to this `left_join` example to merge your features with `vlogger_df` in one single tidy data frame.

## 1.7 Import audiovisual features
Now we can import the audiovisual features from a separate .csv file using `read_delim()`.

In [8]:
# Import audiovisual features df
audiovisual_features_df = read_delim(AudioVisual_file, delim = " ") 
head(audiovisual_features_df, n = 3) # Check the data to see if it is okay

Rows: 404 Columns: 26
── Column specification ────────────────────────────────────────────────────────
Delimiter: " "
chr  (1): vlogId
dbl (25): mean.pitch, sd.pitch, mean.conf.pitch, sd.conf.pitch, mean.spec.en...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


vlogId,mean.pitch,sd.pitch,mean.conf.pitch,sd.conf.pitch,mean.spec.entropy,sd.spec.entropy,mean.val.apeak,sd.val.apeak,mean.loc.apeak,⋯,sd.d.energy,avg.voiced.seg,avg.len.seg,time.speaking,voice.rate,num.turns,hogv.entropy,hogv.median,hogv.cogR,hogv.cogC
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
VLOG1,178.15,0.38358,1.2526,0.45440,3.3674,0.29309,0.82192,0.12429,0.018525,⋯,0.0255970,0.18441,1.3559,0.60796,0.051389,0.44839,7.026606,0.147870,121,198
VLOG3,239.32,0.36474,1.2205,0.41543,3.8150,0.17479,0.64969,0.22731,0.027022,⋯,0.0012289,0.16404,1.0272,0.51374,0.057632,0.50013,4.006787,0.008571,175,164
VLOG5,173.50,0.47636,1.1678,0.50508,3.6949,0.32347,0.65878,0.22253,0.021466,⋯,0.0026112,0.30966,2.2164,0.70205,0.037614,0.31675,7.016616,0.574790,117,156


# 2. Feature extraction from transcript texts

Here you will develop the code that extract features from the transcript texts using `tidytext`. Look at [Introducing Text Analytics](https://www.kaggle.com/code/datasniffer/introducing-text-analytics-big-5-from-text) to see how you should do this. You may also want to copy and paste the `get_lexicon()` function from there to more easily load various lexicons.

It's required to use `tidytext` methods to split and count lexicon keywords as demonstrated in that notebook.

In [9]:
# Helper function to retrieve different lexicons from the 'textdata' package
get_lexicon = function(lexicon_name = names(textdata:::download_functions)) {
    lexicon_name = match.arg(lexicon_name)
    textdata:::download_functions[[lexicon_name]]('.')
    rds_filename = paste0(lexicon_name,'.rds')
    textdata:::process_functions[[lexicon_name]]('.',rds_filename)
    readr::read_rds(rds_filename)
}


## 2.1 Feature extraction from sentiment lexicons

Lexicons help in understanding the opinion or emotion in the text. In our analysis, we will incorporate two lexicons: Afinn and NRC.

Each of these lexicons are based on single words. These lexicons contain English words. These words are assigned scores that capture positive/negative sentiment, also emotions like suprise, joy, anger, sadness, or surprise.

Source: https://afit-r.github.io/sentiment_analysis


### 2.1.1 NRC lexicon

Compute the transcript_features_df dataframe based on the NRC lexicon. The nrc lexicon categorizes words into categories of positive, negative, anger, anticipation, disgust, fear, joy, sadness, surprise, and trust.

In [10]:
# Tokenize transcripts by word
transcripts_tokenized = 
    transcripts_df %>%
    unnest_tokens(token, TEXT, token = 'words')

# Import & remove stopwords
stopwords = get_stopwords()
transcripts_tokenized = 
    transcripts_tokenized %>%
    anti_join(stopwords, by = c(token = "word")) 

# Import "nrc" lexicon
nrc = get_lexicon('nrc')

# Label the sentiments of each word
transcripts_tokenized = 
    transcripts_tokenized %>%
    left_join(nrc,
              by = c(token = 'word'),
              relationship = 'many-to-many')

# Look at results
transcripts_tokenized %>%
    filter(!is.na(sentiment)) %>% 
    head()
names(transcripts_tokenized)

# Compute the sentiment scores and save it in transcript_features_df
transcript_features_df = 
    transcripts_tokenized %>%
    count(vlogId, sentiment) 

# Check results
transcript_features_df %>%
    head()

# Finally, pivot _wider so each vlogger is a column
transcript_features_df = 
    transcript_features_df %>%
    pivot_wider(names_from = sentiment,
                values_from = n,
                values_fill = 0)

# And Change the name of the "NA" column, which are simply words not associated with any sentiments (i.e., neutral)
names(transcript_features_df) = c(names(transcript_features_df)[-length(transcript_features_df)],
                                 "nrc_neutral")

vlogId,filename,token,sentiment
<chr>,<chr>,<chr>,<chr>
VLOG1,../input/bda-2023-profiling-personality/youtube-personality/transcripts/VLOG1.txt,insult,anger
VLOG1,../input/bda-2023-profiling-personality/youtube-personality/transcripts/VLOG1.txt,insult,disgust
VLOG1,../input/bda-2023-profiling-personality/youtube-personality/transcripts/VLOG1.txt,insult,negative
VLOG1,../input/bda-2023-profiling-personality/youtube-personality/transcripts/VLOG1.txt,insult,sadness
VLOG1,../input/bda-2023-profiling-personality/youtube-personality/transcripts/VLOG1.txt,insult,surprise
VLOG1,../input/bda-2023-profiling-personality/youtube-personality/transcripts/VLOG1.txt,trade,trust


[1] "vlogId"    "filename"  "token"     "sentiment"

vlogId,sentiment,n
<chr>,<chr>,<int>
VLOG1,anger,8
VLOG1,anticipation,10
VLOG1,disgust,8
VLOG1,fear,6
VLOG1,joy,7
VLOG1,negative,10


### 2.1.2 AFINN lexicon
Compute the transcript_features_df dataframe based on the AFINN lexicon. The AFINN lexicon assigns a score between -5 and 5 to each word. Negative scores indicate negative sentiment and positive scores indicate positive sentiment (https://afit-r.github.io/sentiment_analysis). 

In [11]:
# Tokenize transcripts by word
transcripts_tokenized = 
    transcripts_df %>%
    unnest_tokens(token, TEXT, token = 'words')

# Import & remove stopwords
stopwords = get_stopwords()
transcripts_tokenized = 
    transcripts_tokenized %>%
    anti_join(stopwords, by = c(token = "word"))

# Import "afinn" lexicon
afinn = get_lexicon('afinn')

# Do an left join an essay token data fame and afinn word list
afinn_token_labeled = left_join(transcripts_tokenized,
                                afinn, by = c(token = 'word'),
                                relationship = 'many-to-many')

# Peek at the result
afinn_token_labeled %>%
    filter(!is.na(value)) %>%
    head()

token_afinn_scores = 
   afinn_token_labeled %>%
    count(`vlogId`, value) 

# Pivot _wider so each vlogger is a column
afinn_value = 
    token_afinn_scores  %>%
    pivot_wider(id_cols = 'vlogId',
                names_from = value,
                values_from = n,
                values_fill = 0)

# Change the name of the "NA" column, which are simply words not associated with any sentiments (i.e., neutral)
names(afinn_value) = c('vlogId','afinn_neg_four','afinn_neg_three',
                       'afinn_neg_two','afinn_one','afinn_two',
                       'afinn_three','afinn_four','afinn_neutral',
                       'afinn_neg_one','afinn_neg_five','afinn_five')

# Check results
afinn_value %>% head(3)

vlogId,filename,token,value
<chr>,<chr>,<chr>,<dbl>
VLOG1,../input/bda-2023-profiling-personality/youtube-personality/transcripts/VLOG1.txt,like,2
VLOG1,../input/bda-2023-profiling-personality/youtube-personality/transcripts/VLOG1.txt,insult,-2
VLOG1,../input/bda-2023-profiling-personality/youtube-personality/transcripts/VLOG1.txt,cool,1
VLOG1,../input/bda-2023-profiling-personality/youtube-personality/transcripts/VLOG1.txt,hell,-4
VLOG1,../input/bda-2023-profiling-personality/youtube-personality/transcripts/VLOG1.txt,like,2
VLOG1,../input/bda-2023-profiling-personality/youtube-personality/transcripts/VLOG1.txt,like,2


vlogId,afinn_neg_four,afinn_neg_three,afinn_neg_two,afinn_one,afinn_two,afinn_three,afinn_four,afinn_neutral,afinn_neg_one,afinn_neg_five,afinn_five
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
VLOG1,1,3,5,4,13,4,1,180,0,0,0
VLOG10,1,1,7,6,8,3,0,228,2,0,0
VLOG100,0,1,1,9,4,4,1,118,1,0,0


## 2.2 Additional text-based features
Now we are going to add four additional features: a proportion of long words, an average sentence length, a proportion of long sentences, and a stop word count.

In [12]:
# 1. PROPORTION OF LONG WORDS (prop_longwords)
# (1a) Create a total word count feature
word_count <- transcripts_tokenized %>%
    group_by(`vlogId`) %>%
    count() %>%
    ungroup() 
# Rename n to word_count and take a look to verify
word_count <- rename(word_count, word_count = n)

# (1b) Create the longwords_count feature 
# First create a vector of the tokens 
vector_tokens <- transcripts_tokenized %>%
    select(`token`) %>%
    unlist() 

# Create a new vector that gives each token a 1 if > 6 letters and 0 if not
longwords_yes_no <- ifelse(nchar(vector_tokens) > 6, 1, 0)

# Now we can create a longwords count feature 
longwords_df <- transcripts_tokenized %>%
    mutate(longwords_yes_no = longwords_yes_no) %>%
    group_by(`vlogId`) %>%
    summarize(longwords_count = sum(longwords_yes_no))

# (1c) Сalculate the proportion of long words
prop_longwords_df <- word_count %>%
    left_join(longwords_df, by = "vlogId") %>%
    mutate(prop_longwords = longwords_count / word_count)  %>%
    select(-c(word_count, longwords_count))

# (1d) Add prop_longwords to the transcripts_tokenized df
transcript_features_df <- transcript_features_df %>%
    left_join(prop_longwords_df, by = "vlogId")

In [13]:
# 2. AVERAGE SENTENCE LENGHT (sent_length)
#  First we have to create a sentence df
sentence_df = 
    transcripts_df %>%
    unnest_tokens(token, TEXT, token = 'sentences')

# Create a vector of the sentences 
vector_sentences <- sentence_df %>%
    select('token')  %>%
    unlist()

# Create a new vector that gives the amount of letters for each sentence
sentence_length <- nchar(vector_sentences)

# Сalculate the average sentence length
sentence_length_df <- sentence_df %>%
    mutate(sent_length = sentence_length) %>%
    group_by(`vlogId`) %>%
    summarize(sent_length = mean(sent_length))

# Add sent_length to the transcripts_tokenized df
transcript_features_df <- transcript_features_df %>%
    left_join(sentence_length_df, by = "vlogId")

In [14]:
# 3. PROPORTION OF LONG SENTENCES (prop_longsent)
# Calculate a total sentence count 
sentence_count <- sentence_df %>%
    group_by(`vlogId`) %>%
    count() %>%
    ungroup()

# Rename n to sentence_count 
sentence_count <- rename(sentence_count, sentence_count = n)

# Add a long sentences (> 167 letters, mean + 1 sd) feature to the df
longsentence_yes_no <- ifelse(nchar(vector_sentences) > 167, 1, 0)

# Calculate a long sentences count
longsentence_df <- sentence_df %>%
    mutate(longsentence_yes_no = longsentence_yes_no) %>%
    group_by(`vlogId`) %>%
    summarize(longsent_count = sum(longsentence_yes_no))

# Calculate a proportion of long sentences 
sentence_count <- sentence_count %>%
    left_join(longsentence_df, by = "vlogId") %>%
    mutate(prop_longsent = longsent_count / sentence_count)  %>%
    select (-c(sentence_count, longsent_count))

## Add prop_longsent to the transcripts_tokenized df
transcript_features_df <- transcript_features_df %>%
    left_join(sentence_count, by = "vlogId")

In [15]:
# 4. STOP WORD COUNT (stopw_count)
# First we get the stop words
stopwords1 <- get_stopwords()
# Let's also add some stopwords we found by looking at the vlog texts and are not included in get_stopwords()
stopwords2 <- tibble(word = c("just", "pretty", "uh", "one", "like", "xxxx",
                              "really", "oh", "yeah", "um", "okay"))
# Put them all in one tibble
stopwords <- bind_rows(stopwords1, stopwords2)

# Count the stopwords
stopwords_df <- transcripts_tokenized %>%
    semi_join(stopwords, by = c(token = "word")) %>%
    group_by(`vlogId`) %>%
    summarize(stopw_count = n())

# Add to the transcripts_tokenized df
transcript_features_df <- transcript_features_df %>%
    left_join(stopwords_df, by = "vlogId")

In [16]:
# Display the final set of transcript features for the first vloggers in `transcript_features_df`
transcript_features_df %>% head(n = 3)

vlogId,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,nrc_neutral,prop_longwords,sent_length,prop_longsent,stopw_count
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>
VLOG1,8,10,8,6,7,10,14,8,5,11,182,0.1800948,58.28947,0.02631579,21
VLOG10,10,19,7,14,9,18,19,8,10,17,206,0.3554688,120.80952,0.23809524,6
VLOG100,0,9,0,2,8,3,11,1,2,9,121,0.1582734,123.08333,0.25000000,36


## 2.3 Merging all features
After computing the features from the transcript texts and storing it in a data frame, we merge it with the `vlogger_df` dataframe and the `audiovisual_features_df`:

In [17]:
# Reload "vlogger_df" and "audivisual_features_df"
vlogger_df = left_join(gender_df, pers_df, by = 'vlogId')
audiovisual_features_df = read_delim(AudioVisual_file, delim = " ") 

# Merge `vlogger_df` with `transcript_features_df` into `vlogger_df`
vlogger_df =
    vlogger_df %>%
    inner_join(transcript_features_df, by = "vlogId")

# Merge `vlogger_df` with `affin_value` into `vlogger_df`
vlogger_df =
    left_join(vlogger_df, afinn_value, by = "vlogId")

# Merge vlogger_df` with `audiovisual_features_df` into `vlogger_df
vlogger_df =
    vlogger_df %>%
    inner_join(audiovisual_features_df, by = "vlogId")

# Look at result
head(vlogger_df, 6)

Rows: 404 Columns: 26
── Column specification ────────────────────────────────────────────────────────
Delimiter: " "
chr  (1): vlogId
dbl (25): mean.pitch, sd.pitch, mean.conf.pitch, sd.conf.pitch, mean.spec.en...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


,vlogId,gender,Extr,Agr,Cons,Emot,Open,anger,anticipation,disgust,⋯,sd.d.energy,avg.voiced.seg,avg.len.seg,time.speaking,voice.rate,num.turns,hogv.entropy,hogv.median,hogv.cogR,hogv.cogC
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,VLOG3,Female,5.0,5.0,4.6,5.3,4.4,1,11,2,⋯,0.0012289,0.16404,1.02720,0.51374,0.057632,0.50013,4.006787,0.008571,175,164
2,VLOG5,Male,5.9,5.3,5.3,5.8,5.5,1,6,1,⋯,0.0026112,0.30966,2.21640,0.70205,0.037614,0.31675,7.016616,0.574790,117,156
3,VLOG6,Male,5.4,4.8,4.4,4.8,5.7,5,11,4,⋯,0.0148060,0.19399,2.53510,0.75993,0.048036,0.29976,3.465855,0.008744,108,179
4,VLOG7,Male,4.7,5.1,4.4,5.1,4.7,12,22,11,⋯,0.0432300,0.56000,1.72040,0.60069,0.024801,0.34916,7.160260,0.285714,135,156
5,VLOG8,Female,NA,NA,NA,NA,NA,3,12,2,⋯,0.0158740,0.16954,0.84412,0.46439,0.056864,0.55015,7.612877,0.418219,123,178
6,VLOG9,Female,5.6,5.0,4.0,4.2,4.9,9,17,8,⋯,0.0006667,0.18044,1.61860,0.67458,0.054172,0.41678,7.032778,0.120711,110,156


# 3. Predictive model

## 3.1 Compare linear models with and without polinomial terms

First, we will fit a baseline Linear Model with all predictions and no polinomial terms.

In [18]:
# Fit full model with all the predictors
fit_mlm = lm(cbind(Extr, Agr, Cons, Emot, Open) ~ ., data = vlogger_df[,-1])

# RMSE (training data)
sqrt(mean(fit_mlm$residuals^2)) 

# Check number of coefficients in the model
length(fit_mlm$coef[,1])

[1] 0.6592562

[1] 53

Now, let's add the second-order polinomial terms for all quantitative variables.

In [19]:
# Create a model with a second-order polinomial term for each of the predictors except gender (full model)
fit_mlm2 = lm(cbind(Extr, Agr, Cons, Emot, Open) ~ . + I(anger^2) + I(anticipation^2) +
                I(disgust^2) + I(fear^2) + I(joy^2) +
                I(negative^2) + I(positive^2) + I(sadness^2) +
                I(surprise^2) + I(trust^2) + I(nrc_neutral^2) +
                I(prop_longwords^2) + I(sent_length^2) + I(prop_longsent^2) + I(stopw_count^2) +
                I(afinn_neg_four^2) + I(afinn_neg_three^2) + I(afinn_neg_two^2) +
                I(afinn_one^2) + I(afinn_two^2) + I(afinn_three^2) +
                I(afinn_four^2) + I(afinn_neutral^2) + I(afinn_neg_one^2) +
                I(afinn_neg_five^2) + I(afinn_five^2) + I(mean.pitch^2) +      
                I(sd.pitch^2) + I(mean.conf.pitch^2) + I(sd.conf.pitch^2) +    
                I(mean.spec.entropy^2) + I(sd.spec.entropy^2) + I(mean.val.apeak^2) +   
                I(sd.val.apeak^2) + I(mean.loc.apeak^2) + I(sd.loc.apeak^2) +     
                I(mean.num.apeak^2) + I(sd.num.apeak^2) + I(mean.energy^2) +      
                I(sd.energy^2) + I(mean.d.energy^2) + I(sd.d.energy^2) +      
                I(avg.voiced.seg^2) + I(avg.len.seg^2) + I(time.speaking^2) +    
                I(voice.rate^2) + I(num.turns^2) + I(hogv.entropy^2) +     
                I(hogv.median^2) + I(hogv.cogR^2) + I(hogv.cogC^2),
              data = vlogger_df[,-1])

# RMSE (training data)
sqrt(mean(fit_mlm2$residuals^2)) 

# Check number of coefficients in the model
length(fit_mlm2$coef[,1])

[1] 0.561021

[1] 104

Now we can compare both models with the `anova()` function.

In [20]:
# The more complex model seems to be better
anova(fit_mlm2, fit_mlm)

,Res.Df,Df,Gen.var.,Pillai,approx F,num Df,den Df,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,220,NA,0.3806552,NA,NA,NA,NA,NA
2,270,50,0.4085149,1.172644,1.348094,250,1100,0.0008815814


Due to the large number of coefficients, there is a large risk of overfitting in the model. We will counteract this by taking a stepwise selection for the coefficients. Specifically, we will use mixed stepwise selection to choose the best predictors for our final models. Stepwise selection is a feature selection technique that will iteratively remove the least informative predictors until it arrives at the most relevant subset of features. This method leads to a model for each trait with a unique set of predictors, optimizing model interpretability and preventing overfitting.
It may be the case however, that the most important predictors are not the same for each of the personality traits. Hence, we will fit 5 different models, one for each personality trait. For each of these, we will perform stepwise selection to reduce the number of predictors and lower the risk of overfitting. For this task, we will use the `stepAIC()` function from the `MASS` package. Before doing so, however, we will try adding some interaction effects between predictors based on theory and substantive reasoning.

## 3.2 Test adding interaction between predictors

We also tried adding different interaction terms to our models, yet none of them led to the considerable decrease in RMSE except for one of the interactions. The code below includes comments detailing the substantive reasoning of why we added these interactions to the baseline model.

In [21]:
## Fitting the model with and without interaction term
# whether longer sentences expressing negative sentiments are associated with higher neuroticism scores
fit_mlm4 <- lm(Emot ~ ., data = vlogger_df[,-c(1,3,4,5,7)])
fit_mlm5 <- lm(Emot ~ . + negative * sent_length, data = vlogger_df[,-c(1,3,4,5,7)])

# whether individuals with higher energy in their voice and more voiced segments tend to be more agreeable
fit_mlm6 <- lm(Agr ~ ., data = vlogger_df[,-c(1,3,5,6,7)])
fit_mlm7 <- lm(Agr ~ . + mean.energy * avg.voiced.seg, data = vlogger_df[,-c(1,3,5,6,7)])

# whether people with more variable pitch and a faster speaking rate tend to be more extraverted
fit_mlm8 <- lm(Extr ~ ., data = vlogger_df[,-c(1,4,5,6,7)])
fit_mlm9 <- lm(Extr ~ . + sd.pitch * voice.rate, data = vlogger_df[,-c(1,4,5,6,7)])

# whether a more central gaze and a higher number of conversational turns are associated with higher agreeableness scores
fit_mlm10 <- lm(Agr ~ ., data = vlogger_df[,-c(1,3,5,6,7)])
fit_mlm11 <- lm(Agr ~ . + hogv.cogC * num.turns, data = vlogger_df[,-c(1,3,5,6,7)]) 

# whether conscientious individuals use longer words when expressing positive sentiments
fit_mlm12 <- lm(Cons ~ ., data = vlogger_df[,-c(1,3,4,6,7)])
fit_mlm13 <- lm(Cons ~ . + positive * prop_longwords, data = vlogger_df[,-c(1,2,4,6,7)])

# whether individuals who exhibit more entropy in their visual behaviors and a central gaze point tend to be more open to experience.
fit_mlm14 <- lm(Open ~ ., data = vlogger_df[,-c(1,3,4,5,6)])
fit_mlm15 <- lm(Open ~ . + hogv.entropy * hogv.cogR, data = vlogger_df[,-c(1,3,4,5,6)])

# whether individuals with a specific vocal peak pattern and more voiced segments tend to be more agreeable
fit_mlm16 <- lm(Agr ~ ., data = vlogger_df[,-c(1,3,5,6,7)])
fit_mlm17 <- lm(Agr ~ . + mean.val.apeak * avg.voiced.seg, data = vlogger_df[,-c(1,3,5,6,7)]) 

# how pitch modulation in speech during expressions of anger relates to neuroticism
fit_mlm18 <- lm(Emot ~ ., data = vlogger_df[,-c(1,3,4,5,7)])
fit_mlm19 <- lm(Emot ~ . + anger * mean.pitch, data = vlogger_df[,-c(1,3,4,5,7)])

# whether individuals who express joy while speaking for longer durations tend to be more extraverted
fit_mlm20 <- lm(Extr ~ ., data = vlogger_df[,-c(1,4,5,6,7)])
fit_mlm21 <- lm(Extr ~ . + joy * time.speaking, data = vlogger_df[,-c(1,4,5,6,7)])

## Change in RMSE after adding interactions
rmse4 <- sqrt(mean(fit_mlm4$residuals^2))
rmse5 <- sqrt(mean(fit_mlm5$residuals^2))
cat("When predicting Neuroticism, the interaction between negative and sent_lengt decreases the RMSE from", rmse4, "to", rmse5, "\n")

rmse6 <- sqrt(mean(fit_mlm6$residuals^2))
rmse7 <- sqrt(mean(fit_mlm7$residuals^2))
cat("When predicting Agreeableness, the interaction between jmean.energy and avg.voiced.seg decreases the RMSE from", rmse6, "to", rmse7, "\n")

rmse8 <- sqrt(mean(fit_mlm8$residuals^2))
rmse9 <- sqrt(mean(fit_mlm9$residuals^2))
cat("When predicting Extraversion, the interaction between sd.pitch and voice.rate decreases the RMSE from", rmse8, "to", rmse9, "\n")

rmse10 <- sqrt(mean(fit_mlm10$residuals^2))
rmse11 <- sqrt(mean(fit_mlm11$residuals^2))
cat("When predicting Agreeableness, the interaction between hogv.cogC and num.turns decreases the RMSE from", rmse10, "to", rmse11, "\n")

rmse12 <- sqrt(mean(fit_mlm12$residuals^2))
rmse13 <- sqrt(mean(fit_mlm13$residuals^2))
cat("When predicting Conscientiousness, the interaction between hogv.cogC and num.turns decreases the RMSE from", rmse12, "to", rmse13, "\n")

rmse14 <- sqrt(mean(fit_mlm14$residuals^2))
rmse15 <- sqrt(mean(fit_mlm15$residuals^2))
cat("When predicting Openness, the interaction between hogv.entropy and hogv.cogR decreases the RMSE from", rmse14, "to", rmse15, "\n")

rmse16 <- sqrt(mean(fit_mlm16$residuals^2))
rmse17 <- sqrt(mean(fit_mlm17$residuals^2))
cat("When predicting Agreeableness, the interaction between mean.val.apeak and avg.voiced.seg decreases the RMSE from", rmse16, "to", rmse17, "\n")

rmse18 <- sqrt(mean(fit_mlm18$residuals^2))
rmse19 <- sqrt(mean(fit_mlm19$residuals^2))
cat("When predicting Neuroticism, the interaction between anger and mean.pitch decreases the RMSE from", rmse18, "to", rmse19, "\n")

rmse20 <- sqrt(mean(fit_mlm20$residuals^2))
rmse21 <- sqrt(mean(fit_mlm21$residuals^2))
cat("When predicting Extraversion, the interaction between joy and time.speaking decreases the RMSE from", rmse20, "to", rmse21, "\n")

When predicting Neuroticism, the interaction between negative and sent_lengt decreases the RMSE from 0.6387021 to 0.6385808 
When predicting Agreeableness, the interaction between jmean.energy and avg.voiced.seg decreases the RMSE from 0.678881 to 0.6786469 
When predicting Extraversion, the interaction between sd.pitch and voice.rate decreases the RMSE from 0.7231238 to 0.7230409 
When predicting Agreeableness, the interaction between hogv.cogC and num.turns decreases the RMSE from 0.678881 to 0.6787878 
When predicting Conscientiousness, the interaction between hogv.cogC and num.turns decreases the RMSE from 0.6399801 to 0.6392766 
When predicting Openness, the interaction between hogv.entropy and hogv.cogR decreases the RMSE from 0.609747 to 0.6095818 
When predicting Agreeableness, the interaction between mean.val.apeak and avg.voiced.seg decreases the RMSE from 0.678881 to 0.6786368 
When predicting Neuroticism, the interaction between anger and mean.pitch decreases the RMSE from 

Based on these results, we will continue modeling with only the inclusion of the interaction between joy and time.speaking for the extraversion.

## Model 1. Extraversion.

In [22]:
# Create table to summarize stepwise selection results
table1 <- data.frame(matrix(NA, 5, 4, dimnames = list(c("Extraversion", "Agreeableness", "Conscientiousness", "Emotional Stability", "Openness"),
                                                      c("RMSE baseline", "Number Predictors Baseline", "RMSE reduced", "Number Predictors Reduced"))))

In [23]:
# Create a model with a second-order polinomial term for each of the predictors except gender (full model)
fit_extr = lm(Extr ~ . + joy * time.speaking - Agr - Cons - Emot - Open + I(anger^2) + I(anticipation^2) +
                I(disgust^2) + I(fear^2) + I(joy^2) +
                I(negative^2) + I(positive^2) + I(sadness^2) +
                I(surprise^2) + I(trust^2) + I(nrc_neutral^2) +
                I(prop_longwords^2) + I(sent_length^2) + I(prop_longsent^2) + I(stopw_count^2) +
                I(afinn_neg_four^2) + I(afinn_neg_three^2) + I(afinn_neg_two^2) +
                I(afinn_one^2) + I(afinn_two^2) + I(afinn_three^2) +
                I(afinn_four^2) + I(afinn_neutral^2) + I(afinn_neg_one^2) +
                I(afinn_neg_five^2) + I(afinn_five^2) + I(mean.pitch^2) +      
                I(sd.pitch^2) + I(mean.conf.pitch^2) + I(sd.conf.pitch^2) +    
                I(mean.spec.entropy^2) + I(sd.spec.entropy^2) + I(mean.val.apeak^2) +   
                I(sd.val.apeak^2) + I(mean.loc.apeak^2) + I(sd.loc.apeak^2) +     
                I(mean.num.apeak^2) + I(sd.num.apeak^2) + I(mean.energy^2) +      
                I(sd.energy^2) + I(mean.d.energy^2) + I(sd.d.energy^2) +      
                I(avg.voiced.seg^2) + I(avg.len.seg^2) + I(time.speaking^2) +    
                I(voice.rate^2) + I(num.turns^2) + I(hogv.entropy^2) +     
                I(hogv.median^2) + I(hogv.cogR^2) + I(hogv.cogC^2),
              data = vlogger_df[,-1])


# RMSE (training data)
table1[1,1] <- sqrt(mean(fit_extr$residuals^2)) 

# Check number of coefficients in the model
table1[1,2] <- length(fit_extr$coef) 

# Run Mixed model selection and save formula
formula <- stepAIC(fit_extr, direction = "both", trace = F)

# Recompute regression model with the paramters dropped
fit_extr_step = lm(formula$call$formula, data = vlogger_df[,-1])

# Summary of final model (after stepwise selection)
summary(fit_extr_step)

# RMSE (training data)
table1[1,3] <- sqrt(mean(fit_extr_step$residuals^2)) 

# Check number of coefficients in the model
table1[1,4] <- length(fit_extr_step$coef) # 43 (vs 105 in the full model)


Call:
lm(formula = formula$call$formula, data = vlogger_df[, -1])

Residuals:
     Min       1Q   Median       3Q      Max 
-1.77996 -0.47021 -0.00459  0.48084  1.64059 

Coefficients:
                      Estimate Std. Error t value Pr(>|t|)    
(Intercept)          5.433e-01  1.000e+00   0.543 0.587524    
genderMale           3.797e-01  1.224e-01   3.102 0.002121 ** 
anger                2.554e-02  1.501e-02   1.701 0.090061 .  
negative            -2.877e-02  1.006e-02  -2.860 0.004559 ** 
nrc_neutral          4.502e-03  1.171e-03   3.846 0.000148 ***
prop_longwords       8.551e+00  3.745e+00   2.284 0.023147 *  
sent_length         -2.088e-02  4.938e-03  -4.228 3.20e-05 ***
stopw_count         -6.627e-03  3.611e-03  -1.835 0.067550 .  
afinn_three          3.273e-02  1.187e-02   2.757 0.006216 ** 
afinn_four           8.084e-02  2.002e-02   4.039 6.95e-05 ***
mean.pitch           2.994e-03  1.084e-03   2.762 0.006134 ** 
sd.pitch             5.405e+00  1.709e+00   3.163 0.001733

## Model 2. Agreeableness.

In [24]:
# Create a model with a second-order polinomial term for each of the predictors except gender (full model)
fit_agr = lm(Agr ~ . - Extr - Cons - Emot - Open + I(anger^2) + I(anticipation^2) +
                I(disgust^2) + I(fear^2) + I(joy^2) +
                I(negative^2) + I(positive^2) + I(sadness^2) +
                I(surprise^2) + I(trust^2) + I(nrc_neutral^2) +
                I(prop_longwords^2) + I(sent_length^2) + I(prop_longsent^2) + I(stopw_count^2) +
                I(afinn_neg_four^2) + I(afinn_neg_three^2) + I(afinn_neg_two^2) +
                I(afinn_one^2) + I(afinn_two^2) + I(afinn_three^2) +
                I(afinn_four^2) + I(afinn_neutral^2) + I(afinn_neg_one^2) +
                I(afinn_neg_five^2) + I(afinn_five^2) + I(mean.pitch^2) +      
                I(sd.pitch^2) + I(mean.conf.pitch^2) + I(sd.conf.pitch^2) +    
                I(mean.spec.entropy^2) + I(sd.spec.entropy^2) + I(mean.val.apeak^2) +   
                I(sd.val.apeak^2) + I(mean.loc.apeak^2) + I(sd.loc.apeak^2) +     
                I(mean.num.apeak^2) + I(sd.num.apeak^2) + I(mean.energy^2) +      
                I(sd.energy^2) + I(mean.d.energy^2) + I(sd.d.energy^2) +      
                I(avg.voiced.seg^2) + I(avg.len.seg^2) + I(time.speaking^2) +    
                I(voice.rate^2) + I(num.turns^2) + I(hogv.entropy^2) +     
                I(hogv.median^2) + I(hogv.cogR^2) + I(hogv.cogC^2),
              data = vlogger_df[,-1])

# RMSE (training data)
table1[2,1] <- sqrt(mean(fit_agr$residuals^2)) 

# Check number of coefficients in the model
table1[2,2] <- length(fit_agr$coef) 

# Run Mixed model selection and save formula
formula <- stepAIC(fit_agr, direction = "both", trace = F)

# Recompute regression model with the paramters dropped
fit_agr_step = lm(formula$call$formula, data = vlogger_df[,-1])

# Summary of final model (after stepwise selection)
summary(fit_agr_step)

# RMSE (training data)
table1[2,3] <- sqrt(mean(fit_agr_step$residuals^2)) 

# Check number of coefficients in the model
table1[2,4] <- length(fit_agr_step$coef) # 65 (vs 104 in the full model)


Call:
lm(formula = formula$call$formula, data = vlogger_df[, -1])

Residuals:
     Min       1Q   Median       3Q      Max 
-1.65697 -0.35502  0.03155  0.38267  1.98027 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)             5.936e+00  1.224e+00   4.851 2.12e-06 ***
genderMale             -3.895e-01  1.184e-01  -3.290 0.001142 ** 
anger                  -2.837e-02  1.793e-02  -1.583 0.114698    
anticipation            3.428e-02  1.715e-02   1.998 0.046736 *  
disgust                -6.544e-02  2.675e-02  -2.446 0.015097 *  
fear                    4.060e-02  1.952e-02   2.080 0.038517 *  
positive                2.515e-02  8.897e-03   2.827 0.005074 ** 
surprise                2.498e-02  1.442e-02   1.732 0.084539 .  
trust                  -3.801e-02  1.630e-02  -2.331 0.020507 *  
prop_longwords          8.689e+00  3.681e+00   2.360 0.019004 *  
sent_length             1.403e-02  8.004e-03   1.753 0.080734 .  
prop_longsent         

## Model 3. Conscientiousness.

In [25]:
# Create a model with a second-order polinomial term for each of the predictors except gender (full model)
fit_cons = lm(Cons ~ . - Extr - Agr - Emot - Open + I(anger^2) + I(anger^2) + I(anticipation^2) +
                I(disgust^2) + I(fear^2) + I(joy^2) +
                I(negative^2) + I(positive^2) + I(sadness^2) +
                I(surprise^2) + I(trust^2) + I(nrc_neutral^2) +
                I(prop_longwords^2) + I(sent_length^2) + I(prop_longsent^2) + I(stopw_count^2) +
                I(afinn_neg_four^2) + I(afinn_neg_three^2) + I(afinn_neg_two^2) +
                I(afinn_one^2) + I(afinn_two^2) + I(afinn_three^2) +
                I(afinn_four^2) + I(afinn_neutral^2) + I(afinn_neg_one^2) +
                I(afinn_neg_five^2) + I(afinn_five^2) + I(mean.pitch^2) +      
                I(sd.pitch^2) + I(mean.conf.pitch^2) + I(sd.conf.pitch^2) +    
                I(mean.spec.entropy^2) + I(sd.spec.entropy^2) + I(mean.val.apeak^2) +   
                I(sd.val.apeak^2) + I(mean.loc.apeak^2) + I(sd.loc.apeak^2) +     
                I(mean.num.apeak^2) + I(sd.num.apeak^2) + I(mean.energy^2) +      
                I(sd.energy^2) + I(mean.d.energy^2) + I(sd.d.energy^2) +      
                I(avg.voiced.seg^2) + I(avg.len.seg^2) + I(time.speaking^2) +    
                I(voice.rate^2) + I(num.turns^2) + I(hogv.entropy^2) +     
                I(hogv.median^2) + I(hogv.cogR^2) + I(hogv.cogC^2),
              data = vlogger_df[,-1])


# RMSE (training data)
table1[3,1] <- sqrt(mean(fit_cons$residuals^2)) 

# Check number of coefficients in the model
table1[3,2] <- length(fit_cons$coef) 

# Run Mixed model selection and save formula
formula <- stepAIC(fit_cons, direction = "both", trace = F)

# Recompute regression model with the paramters dropped
fit_cons_step = lm(formula$call$formula, data = vlogger_df[,-1])

# Summary of final model (after stepwise selection)
summary(fit_cons_step)

# RMSE (training data)
table1[3,3] <- sqrt(mean(fit_cons_step$residuals^2)) 

# Check number of coefficients in the model
table1[3,4] <- length(fit_cons_step$coef) # 45 (vs 104 in the full model)


Call:
lm(formula = formula$call$formula, data = vlogger_df[, -1])

Residuals:
     Min       1Q   Median       3Q      Max 
-2.06618 -0.31199  0.01352  0.37116  1.63544 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)    
(Intercept)           5.741e+00  1.021e+00   5.621 4.63e-08 ***
disgust              -3.695e-02  1.529e-02  -2.416 0.016329 *  
fear                  3.936e-02  1.258e-02   3.129 0.001943 ** 
joy                  -2.430e-02  1.130e-02  -2.151 0.032372 *  
positive              6.383e-02  1.518e-02   4.205 3.53e-05 ***
surprise              4.909e-02  2.640e-02   1.860 0.063965 .  
trust                -3.993e-02  1.896e-02  -2.106 0.036070 *  
nrc_neutral           1.290e-02  6.421e-03   2.009 0.045556 *  
prop_longwords        8.017e+00  3.416e+00   2.347 0.019645 *  
stopw_count          -1.039e-02  4.761e-03  -2.183 0.029883 *  
afinn_neg_four       -2.619e-02  1.614e-02  -1.623 0.105762    
afinn_neg_three      -2.912e-02  1.898e-02  -1

## Model 4. Emotional Stability.

In [26]:
# Create a model with a second-order polinomial term for each of the predictors except gender (full model)
fit_emot = lm(Emot ~ . - Extr - Agr - Cons - Open + I(anger^2) + I(anticipation^2) +
                I(disgust^2) + I(fear^2) + I(joy^2) +
                I(negative^2) + I(positive^2) + I(sadness^2) +
                I(surprise^2) + I(trust^2) + I(nrc_neutral^2) +
                I(prop_longwords^2) + I(sent_length^2) + I(prop_longsent^2) + I(stopw_count^2) +
                I(afinn_neg_four^2) + I(afinn_neg_three^2) + I(afinn_neg_two^2) +
                I(afinn_one^2) + I(afinn_two^2) + I(afinn_three^2) +
                I(afinn_four^2) + I(afinn_neutral^2) + I(afinn_neg_one^2) +
                I(afinn_neg_five^2) + I(afinn_five^2) + I(mean.pitch^2) +      
                I(sd.pitch^2) + I(mean.conf.pitch^2) + I(sd.conf.pitch^2) +    
                I(mean.spec.entropy^2) + I(sd.spec.entropy^2) + I(mean.val.apeak^2) +   
                I(sd.val.apeak^2) + I(mean.loc.apeak^2) + I(sd.loc.apeak^2) +     
                I(mean.num.apeak^2) + I(sd.num.apeak^2) + I(mean.energy^2) +      
                I(sd.energy^2) + I(mean.d.energy^2) + I(sd.d.energy^2) +      
                I(avg.voiced.seg^2) + I(avg.len.seg^2) + I(time.speaking^2) +    
                I(voice.rate^2) + I(num.turns^2) + I(hogv.entropy^2) +     
                I(hogv.median^2) + I(hogv.cogR^2) + I(hogv.cogC^2),
              data = vlogger_df[,-1])

# RMSE (training data)
table1[4,1] <- sqrt(mean(fit_emot$residuals^2)) 

# Check number of coefficients in the model
table1[4,2] <- length(fit_emot$coef) 

# Run Mixed model selection and save formula
formula <- stepAIC(fit_emot, direction = "both", trace = F)

# Recompute regression model with the paramters dropped
fit_emot_step = lm(formula$call$formula, data = vlogger_df[,-1])

# Summary of final model (after stepwise selection)
summary(fit_emot_step)

# RMSE (training data)
table1[4,3] <- sqrt(mean(fit_emot_step$residuals^2)) 

# Check number of coefficients in the model
table1[4,4] <- length(fit_emot_step$coef) # 58 (vs 104 in the full model)


Call:
lm(formula = formula$call$formula, data = vlogger_df[, -1])

Residuals:
     Min       1Q   Median       3Q      Max 
-1.90270 -0.31031  0.05329  0.33973  1.26416 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)             9.526e+00  2.407e+00   3.957 9.75e-05 ***
joy                    -3.180e-02  1.676e-02  -1.897 0.058905 .  
positive                1.898e-02  8.328e-03   2.279 0.023468 *  
surprise                5.751e-02  2.601e-02   2.211 0.027870 *  
prop_longwords          1.354e+01  3.337e+00   4.059 6.49e-05 ***
afinn_neg_four         -4.523e-02  1.587e-02  -2.850 0.004717 ** 
afinn_neg_three        -5.317e-02  1.645e-02  -3.231 0.001389 ** 
afinn_neg_two          -3.414e-02  1.365e-02  -2.501 0.012978 *  
afinn_two               1.645e-02  7.932e-03   2.074 0.039053 *  
afinn_neg_five         -1.775e-01  9.506e-02  -1.867 0.062990 .  
afinn_five             -8.284e-01  3.405e-01  -2.433 0.015627 *  
mean.pitch            

## Model 5. Openness.

In [27]:
# Create a model with a second-order polinomial term for each of the predictors except gender (full model)
fit_open = lm(Open ~ . - Extr - Agr - Cons - Emot + I(anger^2) + I(anticipation^2) +
                I(disgust^2) + I(fear^2) + I(joy^2) +
                I(negative^2) + I(positive^2) + I(sadness^2) +
                I(surprise^2) + I(trust^2) + I(nrc_neutral^2) +
                I(prop_longwords^2) + I(sent_length^2) + I(prop_longsent^2) + I(stopw_count^2) +
                I(afinn_neg_four^2) + I(afinn_neg_three^2) + I(afinn_neg_two^2) +
                I(afinn_one^2) + I(afinn_two^2) + I(afinn_three^2) +
                I(afinn_four^2) + I(afinn_neutral^2) + I(afinn_neg_one^2) +
                I(afinn_neg_five^2) + I(afinn_five^2) + I(mean.pitch^2) +      
                I(sd.pitch^2) + I(mean.conf.pitch^2) + I(sd.conf.pitch^2) +    
                I(mean.spec.entropy^2) + I(sd.spec.entropy^2) + I(mean.val.apeak^2) +   
                I(sd.val.apeak^2) + I(mean.loc.apeak^2) + I(sd.loc.apeak^2) +     
                I(mean.num.apeak^2) + I(sd.num.apeak^2) + I(mean.energy^2) +      
                I(sd.energy^2) + I(mean.d.energy^2) + I(sd.d.energy^2) +      
                I(avg.voiced.seg^2) + I(avg.len.seg^2) + I(time.speaking^2) +    
                I(voice.rate^2) + I(num.turns^2) + I(hogv.entropy^2) +     
                I(hogv.median^2) + I(hogv.cogR^2) + I(hogv.cogC^2),
              data = vlogger_df[,-1])

# RMSE (training data)
table1[5,1] <- sqrt(mean(fit_open$residuals^2)) 

# Check number of coefficients in the model
table1[5,2] <- length(fit_open$coef) 

# Run Mixed model selection and save formula
formula <- stepAIC(fit_open, direction = "both", trace = F)

# Recompute regression model with the paramters dropped
fit_open_step = lm(formula$call$formula, data = vlogger_df[,-1])

# Summary of final model (after stepwise selection)
summary(fit_open_step)

# RMSE (training data)
table1[5,3] <- sqrt(mean(fit_open_step$residuals^2)) 

# Check number of coefficients in the model
table1[5,4] <- length(fit_open_step$coef) # 42 (vs 104 in the full model)


Call:
lm(formula = formula$call$formula, data = vlogger_df[, -1])

Residuals:
     Min       1Q   Median       3Q      Max 
-1.34058 -0.40457  0.00492  0.41031  1.56884 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)    
(Intercept)           3.962e+00  1.027e+00   3.858 0.000142 ***
genderMale            4.185e-01  1.094e-01   3.825 0.000161 ***
anger                 4.852e-02  2.172e-02   2.233 0.026307 *  
negative             -2.931e-02  1.514e-02  -1.936 0.053878 .  
positive              1.473e-02  4.098e-03   3.594 0.000385 ***
surprise             -1.674e-02  1.067e-02  -1.570 0.117574    
prop_longwords        9.068e+00  3.391e+00   2.674 0.007933 ** 
sent_length           1.080e-02  7.362e-03   1.467 0.143432    
prop_longsent        -2.266e+00  1.240e+00  -1.828 0.068567 .  
afinn_four            4.697e-02  1.729e-02   2.717 0.006998 ** 
afinn_neg_five        4.049e-01  2.421e-01   1.673 0.095538 .  
afinn_five           -4.920e-01  3.079e-01  -1

### Table 1. Comparison of baseline models and reduced models after stepwise selection

In [28]:
table1

,RMSE.baseline,Number.Predictors.Baseline,RMSE.reduced,Number.Predictors.Reduced
,<dbl>,<int>,<dbl>,<int>
Extraversion,0.6090791,105,0.6462272,43
Agreeableness,0.5598601,104,0.5723140,65
Conscientiousness,0.5651502,104,0.5928525,45
Emotional Stability,0.5265495,104,0.5448355,58
Openness,0.5399606,104,0.5708363,42


# 4. Making predictions on the test set

## 4.1 The test set

The test set are those `vlogId` that are missing in the personality scores data frame `pers`. They are the rows in `vlogger_df` for which the personality scores are missing:

In [29]:
testset_vloggers = vlogger_df %>% 
    filter(is.na(Extr))

head(testset_vloggers)

,vlogId,gender,Extr,Agr,Cons,Emot,Open,anger,anticipation,disgust,⋯,sd.d.energy,avg.voiced.seg,avg.len.seg,time.speaking,voice.rate,num.turns,hogv.entropy,hogv.median,hogv.cogR,hogv.cogC
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,VLOG8,Female,NA,NA,NA,NA,NA,3,12,2,⋯,0.01587400,0.16954,0.84412,0.46439,0.056864,0.55015,7.612877,0.418219,123,178
2,VLOG15,Male,NA,NA,NA,NA,NA,8,23,3,⋯,0.00169930,0.41882,2.14720,0.71592,0.031669,0.33342,4.199742,0.003863,100,165
3,VLOG18,Male,NA,NA,NA,NA,NA,2,3,1,⋯,0.01954900,0.23584,1.70300,0.69587,0.046223,0.40860,7.107432,0.468896,123,157
4,VLOG22,Female,NA,NA,NA,NA,NA,0,1,1,⋯,0.01786100,0.35004,1.50870,0.42792,0.035558,0.28364,6.713452,0.227571,127,161
5,VLOG28,Male,NA,NA,NA,NA,NA,1,7,1,⋯,0.00062237,0.33089,1.56000,0.52014,0.036923,0.33342,2.880514,0.013793,121,158
6,VLOG29,Female,NA,NA,NA,NA,NA,4,14,3,⋯,0.00377260,0.23339,2.24380,0.63590,0.045721,0.28341,5.036860,0.026627,155,190


## 4.2 Predictions

In [30]:
# Make predictions for each personality score separately
pred_extr = predict(fit_extr_step, new = testset_vloggers[, -c(4,5,6,7)])
pred_agr = predict(fit_agr_step, new = testset_vloggers[, -c(3,5,6,7)])
pred_cons = predict(fit_cons_step, new = testset_vloggers[, -c(3,4,6,7)])
pred_emot = predict(fit_emot_step, new = testset_vloggers[, -c(3,4,5,7)])
pred_open = predict(fit_open_step, new = testset_vloggers[, -c(3,4,5,6)])

# Merge predictions into a data frame
pred = cbind(
    Extr = pred_extr,
    Agr = pred_agr,
    Cons = pred_cons,
    Emot = pred_emot,
    Open = pred_open
    ) %>%
    as.data.frame()

head(pred)

,Extr,Agr,Cons,Emot,Open
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,5.706351,5.225881,4.103566,5.052406,4.946759
2,3.926754,4.401509,4.911829,5.128201,4.561287
3,5.897689,5.167584,4.563131,5.283955,5.677221
4,4.532928,3.096872,3.022277,2.616432,3.723506
5,3.543585,3.790763,4.133493,3.860960,4.145796
6,5.099636,4.409763,4.219668,5.049241,4.453881


In [31]:
# compute output data frame
testset_pred = 
    testset_vloggers[1] %>%
    cbind(pred)

head(testset_pred)

,vlogId,Extr,Agr,Cons,Emot,Open
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,VLOG8,5.706351,5.225881,4.103566,5.052406,4.946759
2,VLOG15,3.926754,4.401509,4.911829,5.128201,4.561287
3,VLOG18,5.897689,5.167584,4.563131,5.283955,5.677221
4,VLOG22,4.532928,3.096872,3.022277,2.616432,3.723506
5,VLOG28,3.543585,3.790763,4.133493,3.860960,4.145796
6,VLOG29,5.099636,4.409763,4.219668,5.049241,4.453881


## 4.3 Writing predictions to file

You need to upload your predictions in .csv file. However, there are multiple columns: `Extr`, `Agr`, `Cons`, `Emot`, `Open`, while Kaggle expects **long format**!

What does long format look like?

- Every prediction on a single line.
- Columns `vlogId` and `pers_axis` to map prediction *vlogger ID* and *personality axis*.

To achieve this, `pivot_longer` to store the personality profile values into a single `value` column:

In [32]:
testset_pred_long <- testset_pred %>% 
    pivot_longer(c(Extr, Agr, Cons, Emot, Open), names_to = 'pers_axis')

head(testset_pred_long)
dim(testset_pred_long)

vlogId,pers_axis,value
<chr>,<chr>,<dbl>
VLOG8,Extr,5.706351
VLOG8,Agr,5.225881
VLOG8,Cons,4.103566
VLOG8,Emot,5.052406
VLOG8,Open,4.946759
VLOG15,Extr,3.926754


[1] 400   3

According to the competition's [Evaluation instructions](https://www.kaggle.com/competitions/bda-2023-bigfive/overview/evaluation), Kaggle expects file with two colums: `Id` and `Expected`.
  
The [Evaluation instructions](https://www.kaggle.com/competitions/bda-2023-bigfive/overview/evaluation) specifies we need to encode the `Agr` prediction for `VLOG8` as `VLOG8_Agr` in the `Id` column. To achieve this use `unite()` function of `dplyr`.

`unite()` take:

- a data frame as its first argument (implicitely passed by the piping operator `%>%`)
- the name of new column as its second argument (`Id` below)
- all extra arguments (`vlogId` and `pers_axis` below) are concatenated with an underscore in between

Then write the resulting data frame to a .csv file.

In [33]:
# Obtain the right format for Kaggle
testset_pred_final <- testset_pred_long %>%
    unite(Id, vlogId, pers_axis) %>%
    rename(Expected = value)

# Check if we succeeded
head(testset_pred_final)

Id,Expected
<chr>,<dbl>
VLOG8_Extr,5.706351
VLOG8_Agr,5.225881
VLOG8_Cons,4.103566
VLOG8_Emot,5.052406
VLOG8_Open,4.946759
VLOG15_Extr,3.926754


In [34]:
# Write to csv
write_csv(testset_pred_final, file = "predictions.csv")

# Check if the file was written successfully.
dir()

[1] "__MACOSX"                "__notebook__.ipynb"     
[3] "afinn.rds"               "imm6010.zip"            
[5] "NRC-Emotion-Lexicon"     "NRC-Emotion-Lexicon.zip"
[7] "nrc.rds"                 "predictions.csv"

## Division of Labor

Alex: adding extra text-based features, improving code readability, adding interaction effects and assesing their impact.

Roy: adding NRC lexicon, running regression & stepwise selection, improving code readability and formatting as a whole.

Titus: adding AFINN lexicon, adding most explanatory texts, searching for references, improving document presentation as a whole.

Together: brainstorming ideas to improve our model, final revisions and proofreading.

# References

Huntington, C. (n.d.). Big Five Personality Traits: Definition & Theory. Retrieved from https://www.berkeleywellbeing.com/big-five-personality-traits.html

Schmitt, D. P., Realo, A., Voracek, M., & Allik, J. (2008). Why can't a man be more like a woman? Sex differences in Big Five personality traits across 55 cultures. Journal of Personality and Social Psychology, 94(1), 168–182. https://doi.org/10.1037/0022-3514.94.1.168

Text Mining: Sentiment Analysis. (n.d.). Retrieved from https://afit-r.github.io/sentiment_analysis